## Assignment 01 - Exercise 1

Implement a function linearThresholdUnit that takes a list of x-values and a list of wvalues,calculates the weighted sum and returns 1 if that sum is greater than or equal – otherwise 0.

In [6]:
def linearThresholdUnit(x_values,w_values):
    weightedsum = sum(x*w for x,w in zip(x_values,w_values))
    if weightedsum > 0:
        return 1
    else:
        return 0

In [7]:
x = [1,0,1]
w=[2,-2,-1]
print(linearThresholdUnit(x,w))

1


## Assignment 02 - Exercise 2

Implement the spam detection classifier (slide 29 in ”Lecture 1 - Introduction”) in Python. You can do this by following the steps below:
-Transfer the data set on slide 29 into an appropiate Python data structure
-Implement the classifier on slide 29 that takes into account the three features “Occurences Bank”, “Occurences 1 000 000” and “Occurences Best regards”. For each input, the output should be either “True” or “False”. 
Hint: You may be able to reuse some of the code you already implemented in assignment 1.
-Write Python to calculate the accuracy of the clasifier’s predictions.
-Play around with different weights and see if you can further improve the accuracy.

In [8]:
datadic = {"dataset1":[1,0,1],"dataset2":[1,0,2],"dataset3":[1,2,0],"dataset4":[0,1,1],"dataset5":[0,0,1]}
resultdic = {"dataset1":["True"],"dataset2":["True"],"dataset3":["False"],"dataset4":["False"],"dataset5":["False"]}
weights = [2,-2,-1]

t=0
f=0

def relu_function(x, w):
    relu = sum(xi * wi for xi, wi in zip(x, w))
    return relu

for key in datadic:
    value= relu_function(datadic[key], weights)
    result = resultdic[key][0]
    if value >= 0 and result == "True":
        print(f"Value:{value} --> Spam \n Correct")
        t+=1
    elif value >= 0 and result == "False":
        print(f"Value:{value} --> Spam \n False")
        f+=1
    elif value <= 0 and result == "False":
        print(f"Value:{value} --> no Spam \n Correct")
        t+=1
    elif value <= 0 and result == "True":
        print(f"Value:{value} --> no Spam \n False")
        f+=1

accuracy = t/(t+f)*100
print(f"The accuracy of the classifier is at {accuracy}%")

Value:1 --> Spam 
 Correct
Value:0 --> Spam 
 Correct
Value:-2 --> no Spam 
 Correct
Value:-3 --> no Spam 
 Correct
Value:-1 --> no Spam 
 Correct
The accuracy of the classifier is at 100.0%


## Assignment 03 - Exercise 1
Take another look at exercise 1 of assignment 2. In this exercise, your task was to implement the spam detection model from the lecture using Python. Since you are now familiar with NumPy, try to substitute as much of the pure Python code with improved NumPy code. Place special focus on lists.

In [9]:
import numpy as np
data = [[1,0,1],[1,0,2],[1,2,0],[0,1,1],[0,0,1]]
weights = [2,-2,-1]
labels = [True,True,False,False,False]

sums = np.sum(np.multiply(data,weights),axis=1)
results = sums >= 0
matches = np.sum(results == labels)

print(f"Accuracy: {matches/len(data)*100}%")

Accuracy: 100.0%


## Assignment 04 - Exercise 2
Imagine you are a newspaper deliverer that encounters a number of dogs on your tour. Some of them (try to) bite, some of then only bark. The dogs are described by the following binary features: Heavy, Smelly, Big and Growling. Consider the following set of examples:
| Heavy | Smelly | Big | Growling | Action |
|----------|----------|----------|----------|----------|
| No | No | No | No | Bark |
| No | No | Yes | No | Bark |
| Yes | Yes | No | Yes | Bark |
| Yes | No | No | Yes | Bite |
| No | Yes | Yes | No | Bite |
| No | No | Yes | Yes | Bite |
| No | No | No | Yes | Bite |
| Yes | Yes | No | No | Bite |

Round all values within this exercise back to 4 decimal places. Note that we’ve prepared the
dataset as a NumPy array in dogtree.ipynb. You may use this notebook for your calculations or
work out the answers manually – depending on your liking.


### Fullsolution in Assignment04 dogtree.ipynb

#### Task a) What is the entropy of the target value ’Action’ in the whole dataset?

In [10]:
import pandas as pd
import numpy as np
import math

# read dog dataset, No is encoded as 0, Yes as 1, 
# bite refers to class 1, bark to class 0

training_data = pd.read_csv("Assignment04\dogs.csv") 
def entropy(values):
        # could be empty  while trying for a split
        if values.size == 0:
            return 0
        
        ratio_ones = sum(values) / len(values)
        ratio_zeros = 1 - ratio_ones

        pd = [ratio_ones, ratio_zeros]
        if ratio_ones == 0 or ratio_zeros == 0:
            return 0
        return - sum( [ p_i * np.log2(p_i) for p_i in pd] )
print(entropy(training_data["Action"].to_numpy()))



0.954434002924965


#### Entropy for Action: 0.9544

#### Task b) Which attribute would the ID3 algorithm choose to use for the root of the tree? What is its information gain?

In [11]:
igains =[]
attributes = []
for col in training_data.columns:
    igains.append(entropy((training_data[col] == 0).to_numpy())-entropy(training_data["Action"].to_numpy()))
    attributes.append(col)
print(max(zip(igains,attributes)))

(0.04556599707503495, 'Growling')


#### --> Growling, information gain = 0.0455

#### Task c) Draw the full decision tree that would be learned for this data using ID3 without pruning.

In [17]:
class Node:
    
    def entropy(self, values):
        # could be empty  while trying for a split
        if values.size == 0:
            return 0
        
        ratio_ones = sum(values) / len(values)
        ratio_zeros = 1 - ratio_ones

        pd = [ratio_ones, ratio_zeros]
        if ratio_ones == 0 or ratio_zeros == 0:
            return 0
        return - sum( [ p_i * np.log2(p_i) for p_i in pd] )
    
    def __init__(self, data, ancestor_features):
        # pure if there are all ones or all zeros 
        self.pure = sum(data["Action"]) == len(data) or  sum(data["Action"]) == 0
        self.split_feature = None
        self.data = data
        self.base_entropy = self.entropy(data["Action"].to_numpy())
        self.children = None
        # since we're using categorical (binary), each features should at most be used once in any branch
        self.ancestor_features = ancestor_features 
        # You can use these ancestor_features to get a nice identation:
        print(" "*len(self.ancestor_features) + f"{data}")
        
    def split(self):
        # implement the split here
        features = [col for col in self.data.columns if col not in self.ancestor_features and col != "Action"]
        max_info_gain = -1
        best_feature = None
        
        for feature in features:
            for value in self.data[feature].unique():
                subset = self.data[self.data[feature] == value]
                entropy = self.entropy(subset["Action"].to_numpy())
                info_gain = self.base_entropy - entropy
                
                if info_gain > max_info_gain:
                    max_info_gain = info_gain
                    best_feature = feature
                    best_threshold = value
        self.split_threshold = best_threshold
        if best_feature is not None:
            self.split_feature = best_feature
            self.children = []
            
            for value in self.data[best_feature].unique():
                subset = self.data[self.data[best_feature] == value]
                child = Node(subset, self.ancestor_features + [best_feature])
                self.children.append(child)
    def train(self):        
        if not self.pure:
            self.split()
            for child in self.children:
                # assume that we can get nodes pure by looking at our features
                # having exactly the same features but two different classes would not work here
                child.train()
                # now child is either pure or split into nodes that are eventually pure
# start training with a root node consisting of all data
    def predict(self, new_data):
        if self.pure:
            return self.data["Action"].iloc[0]
        else:
            # Andernfalls entscheide anhand des geteilten Merkmals und Schwellenwerts
            feature_value = new_data[self.split_feature]
            if feature_value <= self.split_threshold:
                # Wenn der Wert kleiner oder gleich dem Schwellenwert ist, gehe zum linken Kindknoten
                return self.children[0].predict(new_data)
            else:
                # Andernfalls gehe zum rechten Kindknoten
                return self.children[1].predict(new_data)
root = Node(training_data, [])
root.train()


   Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
2      1       1    0         1       0
3      1       0    0         1       1
4      0       1    1         0       1
5      0       0    1         1       1
6      0       0    0         1       1
7      1       1    0         0       1
    Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
4      0       1    1         0       1
7      1       1    0         0       1
    Heavy  Smelly  Big  Growling  Action
2      1       1    0         1       0
3      1       0    0         1       1
5      0       0    1         1       1
6      0       0    0         1       1
     Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
4      0       1    1         0       1
     Heavy  Smelly  Big  Growling  Action
7      1       1    0         0   

#### Task d) Suppose three new dogs appear in your round as listed in the table below. Classify them using the decision tree from the previous question. 
#### What is the accuracy of your tree on this test set?

| Name | Heavy | Smelly | Big | Growling | Action |
|----------|----------|----------|----------|----------|----------|
| Buster | Yes | No | Yes | Yes | Bark |
| Pluto | No | No | No | Yes | Bark |
| Zeus | Yes | Yes | No | No | Bite |

In [13]:
test_data = [{"Heavy":1,"Smelly":0,"Big":1,"Growling":1}, {"Heavy": 0, "Smelly": 0, "Big": 0, "Growling": 1},{"Heavy": 1, "Smelly": 1, "Big": 0, "Growling": 0}]
for data_point in test_data:
    print("Prediction:"+ str(root.predict(data_point)))

Prediction:0
Prediction:0
Prediction:1


#### --> 100% Accuracy

#### Task e) Repeat the steps of the previous four subtasks using the Gini coefficient as splitting criteria.
#### Are there differences between the resulting trees and their performance on the above test set? Think about the reasons.

In [18]:
#rewriting class with gini instead of entropy
class Node:
    
    def gini(self,values):
        base=len(values)
        if values.size == 0:
            return 0
        count0=sum([1 for i in values if i == 0])
        count1=base-count0
        p0=count0 / base
        p1=count1 / base
        result = 1-(p0**2+p1**2)
        return result
    
    def __init__(self, data, ancestor_features):
        # pure if there are all ones or all zeros 
        self.pure = sum(data["Action"]) == len(data) or  sum(data["Action"]) == 0
        self.split_feature = None
        self.data = data
        self.base_gini = self.gini(data["Action"].to_numpy())
        self.children = None
        # since we're using categorical (binary), each features should at most be used once in any branch
        self.ancestor_features = ancestor_features 
        # You can use these ancestor_features to get a nice identation:
        print(" "*len(self.ancestor_features) + f"{data}")
        
    def split(self):
        # implement the split here
        features = [col for col in self.data.columns if col not in self.ancestor_features and col != "Action"]
        max_info_gain = -1
        best_feature = None
        
        for feature in features:
            for value in self.data[feature].unique():
                subset = self.data[self.data[feature] == value]
                gini = self.gini(subset["Action"].to_numpy())
                info_gain = self.base_gini - gini
                
                if info_gain > max_info_gain:
                    max_info_gain = info_gain
                    best_feature = feature
                    best_threshold = value
        self.split_threshold = best_threshold
        if best_feature is not None:
            self.split_feature = best_feature
            self.children = []
            
            for value in self.data[best_feature].unique():
                subset = self.data[self.data[best_feature] == value]
                child = Node(subset, self.ancestor_features + [best_feature])
                self.children.append(child)
    def train(self):        
        if not self.pure:
            self.split()
            for child in self.children:
                # assume that we can get nodes pure by looking at our features
                # having exactly the same features but two different classes would not work here
                child.train()
                # now child is either pure or split into nodes that are eventually pure
# start training with a root node consisting of all data
    def predict(self, new_data):
        if self.pure:
            return self.data["Action"].iloc[0]
        else:
            # Andernfalls entscheide anhand des geteilten Merkmals und Schwellenwerts
            feature_value = new_data[self.split_feature]
            if feature_value <= self.split_threshold:
                # Wenn der Wert kleiner oder gleich dem Schwellenwert ist, gehe zum linken Kindknoten
                return self.children[0].predict(new_data)
            else:
                # Andernfalls gehe zum rechten Kindknoten
                return self.children[1].predict(new_data)
root_gini = Node(training_data, [])
root_gini.train()

   Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
2      1       1    0         1       0
3      1       0    0         1       1
4      0       1    1         0       1
5      0       0    1         1       1
6      0       0    0         1       1
7      1       1    0         0       1
    Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
4      0       1    1         0       1
7      1       1    0         0       1
    Heavy  Smelly  Big  Growling  Action
2      1       1    0         1       0
3      1       0    0         1       1
5      0       0    1         1       1
6      0       0    0         1       1
     Heavy  Smelly  Big  Growling  Action
0      0       0    0         0       0
1      0       0    1         0       0
4      0       1    1         0       1
     Heavy  Smelly  Big  Growling  Action
7      1       1    0         0   

In [15]:
for data_point in test_data:
    print("Prediction:"+ str(root_gini.predict(data_point)))


Prediction:0
Prediction:0
Prediction:1


#### --> 100% Accuracy